In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
#nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
import string

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from scipy import stats
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc, roc_curve
from sklearn.metrics import f1_score, classification_report

from sklearn import svm
import gensim.models as g
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import auc, roc_curve, roc_auc_score

import gensim.downloader as api

from nltk.corpus import wordnet

import gensim.models as g
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from tqdm import tqdm
from sklearn import utils

In [2]:
data = pd.read_csv('/Users/daviddeng8/Documents/Projects/affinity/data/preprocessed_cleaned_body.csv')

In [3]:
data.head()

,Body,Bias,cleaned_body
0,Abortion rights advocates have asked the U.S. ...,1.67,abortion right advocate asked u supreme court ...
1,A federal appeals court rejected the most dire...,0.67,federal appeal court rejected direct constitut...
2,As part of the Trump administration's effort t...,-2.75,part trump administration effort slow migrant ...
3,"President Donald Trump and ""the Trump of the T...",-4.33,president donald trump trump tropic brazilian ...
4,"U.S Senator Elizabeth Warren, who is competing...",-10.00,u senator elizabeth warren competing democrati...


In [4]:
right = data.loc[data['Bias'] >= 5]
neutral = data.loc[(data['Bias'] < 5) & (data['Bias'] > -5)]
left = data.loc[data['Bias'] <= 5]

In [5]:
cat_df = pd.concat([right, neutral, left])
cat_df.head()

,Body,Bias,cleaned_body
13,President Donald Trump has vetoed a resolution...,9.75,president donald trump ha vetoed resolution st...
155,Asked by if he believes President Trump's emer...,23.75,asked belief president trump emergency declara...
156,You May Like\nThese 10 Shawshank Redemption Fa...,8.50,may like shawshank redemption fact insane hive...
157,"NEW: This Germantown, Md. man tells he was att...",14.00,new germantown md man tell wa attacked saturda...
158,“A little as two to three milligrams of fentan...,16.75,“a little two three milligram fentanyl induce ...


In [6]:
cat_df.shape

(2343, 3)

In [7]:
#model = g.Doc2Vec.load('/Users/daviddeng8/Desktop/enwiki_dbow/doc2vec.bin')

In [8]:
def label_sentences(corpus, label_type):
    labeled = []
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(TaggedDocument(v.split, [label]))
    return labeled

In [9]:
X_train, X_test, y_train, y_test = train_test_split(cat_df.cleaned_body, cat_df.Bias, test_size=0.2)
X_train

628     may think it’s fun walk shoe celebrity — photo...
103     nation ha moved history result donald trump’s ...
1287    venezuela bracing potential faceoff weekend ni...
1005    started selling race car equipment year ago ch...
965     “all right” anchor alisyn camerota said “get u...
                              ...                        
840     sen lindsey graham rsc chuck grassley riowa ro...
1368    several state already filed lawsuit prevent ci...
200     chemical plant near houston ha burning since s...
473     rep lee zeldin rny continued sound thursday’s ...
958     reutersjames lawler duggan u politician side a...
Name: cleaned_body, Length: 1874, dtype: object

In [10]:
X_train = label_sentences(X_train, 'Train')
X_test = label_sentences(X_test, 'Test')
all_data = X_train + X_test

In [11]:
X_train[:2]

[TaggedDocument(words=<built-in method split of str object at 0x7fa10f8b4eb0>, tags=['Train_0']),
 TaggedDocument(words=<built-in method split of str object at 0x7fa10f58b800>, tags=['Train_1'])]

In [12]:
len(all_data)

2343

In [13]:
def get_vectors(model, corpus_size, vectors_size, vectors_type):
    """
    Get vectors from trained doc2vec model
    :param doc2vec_model: Trained Doc2Vec model
    :param corpus_size: Size of the data
    :param vectors_size: Size of the embedding vectors
    :param vectors_type: Training or Testing vectors
    :return: list of vectors
    """
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = model.docvecs[prefix]
    return vectors

In [16]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_data)])

100%|██████████| 2343/2343 [00:00<00:00, 786872.79it/s]


TypeError: object of type 'builtin_function_or_method' has no len()

In [15]:
%%time
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(all_data)]), total_examples=len(all_data), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 2343/2343 [00:00<00:00, 620485.81it/s]


RuntimeError: you must first build vocabulary before training the model

In [17]:
train_vectors_dbow = get_vectors(model_dbow, len(X_train), 300, 'Train')
test_vectors_dbow = get_vectors(model_dbow, len(X_test), 300, 'Test')

KeyError: "tag 'Train_0' not seen in training corpus/invalid"

In [ ]:
train_vectors_dbow

In [ ]:
#frame = { 'vector': train_vectors_dbow}
#train_vectors_dbow = pd.DataFrame(train_vectors_dbow)
#test_vectors_dbow = pd.DataFrame(test_vectors_dbow)
mean_train = [np.mean(text) for text in train_vectors_dbow]
mean_test = [np.mean(text) for text in test_vectors_dbow]
mean_train

In [ ]:
temp_series = pd.Series(mean_train)
frame = {'vector' : temp_series}
mean_train = pd.DataFrame(frame)
temp_series = pd.Series(mean_test)
frame = {'vector' : temp_series}
mean_test = pd.DataFrame(frame)

In [ ]:
classification_model = svm.SVC()
classification_model.fit(mean_train, y_train)
y_pred = classification_model.predict(mean_test)
print("Accuracy score: ", accuracy_score(y_test, y_pred))